__体素缺陷人工标注代码__

In [ ]:
import numpy as np
import scipy.io as io

data = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = data['defectdata']
data = np.array(data)
index = io.loadmat('../../Matlab_files/index.mat')
index = index['indexdata1']
index = np.array(index)
index_array = []
for i in range(len(index)):
    #提取一行进行处理
    deal_index = index[i]

    
    #增加维度，调整x、y、z列的顺序
    deal_index = deal_index[np.newaxis,:]
    deal_index = deal_index[:,[2, 0, 1]]

    #matlab中坐标转化到python中坐标
    new_x = 120 - deal_index[:,[0]]
    new_z = 120 - deal_index[:,[1]]
    new_y =  -deal_index[:,[2]]
    
    #将坐标存储到坐标列表中
    temp_array = [new_x, new_z, new_y]
    temp_array = np.squeeze(temp_array)
    index_array.append(temp_array)

index_array = np.array(index_array)
print(index_array)

for t in range(len(index_array)):
    embed_index = index_array[t]
    print(embed_index)
    x = embed_index[0]
    z = embed_index[1]
    y = embed_index[2]
    data[x][z][y] = 1

io.savemat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat',{'defectdata':data})


__数据集切割代码__

In [ ]:
import numpy as np
import scipy.io as io
import os

def cutmodule(matdata, saveindex, signal):
    if signal == True:
        data = matdata['data']
        print(data.shape)
        print('****************执行数据尺寸修改****************')
        nelx, nely, nelz = 120, 120, 120
        if (nelx - data.shape[0])%2 !=0:
            data = np.pad(data, ((1, 0), (0, 0), (0 ,0)))
        if (nelz - data.shape[1])%2 !=0:
            data = np.pad(data, ((0, 0), (1, 0), (0 ,0)))
        if (nely - data.shape[2])%2 !=0:
            data = np.pad(data, ((0, 0), (0, 0), (1 ,0)))
        data = np.pad(data, (((nelx - data.shape[0])//2, (nelx - data.shape[0])//2),
                        ((nely - data.shape[1])//2, (nely - data.shape[1])//2),
                        ((nelz - data.shape[2])//2, (nelx - data.shape[2])//2)))
        io.savemat(os.path.join(saveindex,'inputdata.mat'),{'inputdata':data})
        tempdata = np.zeros((20, 20, 20))
        for k in range(0, data.shape[2], 20):               #沿y轴切割
            for j in range(0, data.shape[1], 20):           #沿z轴切割
                for i in range(0, data.shape[0], 20):       #沿x轴切割
                    global m
                    m += 1
                    tempdata = data[i:i+20, j:j+20, k:k+20]
                    # np.set_printoptions(threshold=np.inf)
                    np.save(os.path.join(saveindex,'%d.npy'%m),tempdata)
                    print('第：',m,'次')
                    # print(tempdata)
                    tempdata = np.zeros((20, 20, 20))
    else:
        data = matdata['defectdata']
        print(data.shape)
        print('****************生成标签数据集****************')
        tempdata = np.zeros((20, 20, 20))
        for k in range(0, data.shape[2], 20):               #沿y轴切割
            for j in range(0, data.shape[1], 20):           #沿z轴切割
                for i in range(0, data.shape[0], 20):       #沿x轴切割
                    global s
                    s += 1
                    tempdata = data[i:i+20, j:j+20, k:k+20]
                    # np.set_printoptions(threshold=np.inf)
                    np.save(os.path.join(saveindex,'%d.npy'%s),tempdata)
                    print('第：',s,'次')
                    # print(tempdata)
                    tempdata = np.zeros((20, 20, 20))

__非零矩阵标号检测__

In [ ]:
import numpy as np
import os
import glob

def nozerosdetect(data_index):

    global m
    global s
    n = 0
    zeros_array = []
    array = np.arange((m-215), (m+1))
    
    for i in range((m-215),(m+1)):
        path = os.path.join(data_index, str(i)+'.npy')
        data = np.load(path)
        if np.all(data == 0):
            n += 1
            zeros_array.append(i)
        else:
            zeros_array.append(0)

    print('一共',n,'个全零矩阵')

    zeros_array = np.array(zeros_array)
    nozeros_array = array - zeros_array
    np.save(os.path.join(data_index, 'detectmatrix.npy'), nozeros_array)

__主函数__

In [ ]:
import numpy as np
import os
import scipy.io as io
import glob

folder_address = '../../Matlab_files/dataset/originalmatdata/dataset/dataset(05.25)/input/'
mat_address = '../../Matlab_files/dataset/originalmatdata/dataset/matdata/input/'
# folder_address = '../Matlab_files/dataset/originalmatdata/dataset/dataset(05.19)/input/'
# mat_address = '../../Matlab_files/dataset/originalmatdata/matdata/input/'
#test
# folder_address = '../Matlab_files/dataset/originalmatdata/testdataset/input/'
# mat_address = '../Matlab_files/dataset/originalmatdata/matdata/input/'

matdata_index = glob.glob(os.path.join(mat_address,'*.mat')) 

m = 0
s = 0
for t in range(0, len(matdata_index)):

    print('读取第',t,'个数据')

    #输入数据集生成
    os.makedirs(os.path.join(folder_address,str(t)))
    inputpath = os.path.join(mat_address,str(t)+'.mat')      #按顺序读取输入数据
    inputmatdata = io.loadmat(inputpath)
    saveinputindex = os.path.join(folder_address,str(t))
    cutmodule(inputmatdata, saveinputindex, signal=True)
    data_index = os.path.join('../../Matlab_files/dataset/originalmatdata/dataset/dataset(05.25)/input/',str(t)+'/')
    nozerosdetect(data_index)
    
    
    #标签数据集生成
    label_folder_address = folder_address.replace('input/','label/')   #文件夹生成地址
    os.makedirs(os.path.join(label_folder_address,str(t)))             #生成对应模型的存储文件夹
    labelpath = inputpath.replace('input/','label/')                   #标签数据读取路径
    savelabelindex = os.path.join(label_folder_address,str(t))
    labelmatdata = io.loadmat(labelpath)
    cutmodule(labelmatdata, savelabelindex, signal=False)
    label_data_index = data_index.replace('input/','label/')
    nozerosdetect(label_data_index)
    print('*'*30)
    


In [ ]:
import numpy as np

data = np.load('../../Matlab_files/dataset/originalmatdata/dataset/dataset(05.25)/input/21/4631.npy')
print(np.all(data==0))
labeldata = np.load('../../Matlab_files/dataset/originalmatdata/dataset/dataset(05.25)/label/21/4631.npy')
print(np.all(labeldata==0))

In [ ]:
import numpy as np
import scipy.io as io
# data = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/input/0/detectmatrix.npy')
# print(data)
inputdata = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/input/81.npy')
print(inputdata.shape)
io.savemat('./testdata.mat',{'inputdata':inputdata})
labeldata = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/label/81.npy')
io.savemat('./labeldata.mat',{'labeldata':labeldata})

In [ ]:
import numpy as np
import scipy.io as io

data = io.loadmat('../Matlab_files/dataset/originalmatdata/matdata/input/0.mat')
data = data['data']
print(data.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = np.load('../Matlab_files/dataset/originalmatdata/dataset/label/45/detectmatrix.npy')
print(data)
slicedata = np.load('../Matlab_files/dataset/originalmatdata/dataset/label/45/9807.npy')
np.set_printoptions(threshold=np.inf)
# print(slicedata)
print(np.all(slicedata == 0))
mycolormap = plt.get_cmap('winter')
slicedata_max = slicedata.max()
relativevalue = np.round(slicedata / slicedata_max, 1)
colorsvalues = mycolormap(relativevalue)
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(projection='3d')
ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')
ax.set_zlabel('Z Axis')
ax.set_xlim(0, 20)
ax.set_ylim(0, 20)
ax.set_zlim(10, -10)
ax.voxels(slicedata, facecolors=colorsvalues,edgecolors = 'k', shade = False)

__Mask转One-hot__

In [ ]:
import torch
index = torch.randint(7 ,size=(1, 1, 7, 7)).long()
print(index)
x_onehot = torch.zeros(1, 7, 7, 7).long()
x_onehot.scatter_(1, index, 1).float()
print(x_onehot.numpy())

__defectdata实现Mask转One-hot__

In [ ]:
import scipy.io as io
import numpy as np
import torch
matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = matdata['defectdata']
np.set_printoptions(threshold=np.inf)
# print(data)
data = data[np.newaxis,np.newaxis,:,:,:]
data = torch.from_numpy(data).long()
print(data.shape)
np.set_printoptions(threshold=np.inf)
output = torch.zeros(1, 3, 120, 120, 120).long()            #output的channel数一定要等于类别数，同时也要等于mask中的类别数，因为scatter相当于是Mask
                                                            #拆成了与类别数相对应的One-hot，One-hot每个通道对应一个类别，总之：
                                                            #One-hot channel数 = 类别数 = mask中数字类别个数
print(output.shape)
output = output.scatter(1, data, 1)
print(output)

In [ ]:
import scipy.io as io
import numpy as np

# matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
matdata = io.loadmat('./test.mat')
data = matdata['data']
print(data.shape)
b = np.nonzero(data)
b = np.array(b)
print(b)
print(type(b))

for i in range(b.shape[1]):
    print('第',i,'个数据')
    print(data[(b[0])[i]][(b[1])[i]][(b[2])[i]])
    m = i
print(m)

In [ ]:
import torch

index = torch.randint(3, (1, 1, 5, 5)).long()
print('index:\n',index)

ouput_onehot = torch.zeros(1, 3, 5, 5).long()
ouput_onehot = ouput_onehot.scatter(1, index, 1)
print('output_onehot:\n',ouput_onehot)


In [ ]:
import scipy.io as io
import numpy as np

matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = matdata['defectdata']
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if data[i][j][k] != 0:
                data[i][j][k] = 1
            
io.savemat('test.mat',{'data':data})

In [ ]:
import numpy as np
import scipy.io as io
import torch
matdata = io.loadmat('./test.mat')
data = matdata['data']
data = data[np.newaxis,np.newaxis,:,:,:]
data = torch.from_numpy(data).long()

print(data.shape)

output_onehot = torch.zeros(1, 2, 120, 120, 120)
output_onehot = output_onehot.scatter(1 ,data, 1)
print(output_onehot)

In [ ]:
import numpy as np

data_index = np.load('../../Matlab_files/dataset/originalmatdata/dataset/input/0/detectmatrix.npy')
data = np.load('../../Matlab_files/dataset/originalmatdata/dataset/input/0/82.npy')
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if data[i][j][k] != 0:
                data[i][j][k] = 1
np.set_printoptions(threshold=np.inf)
# print(data)
# print(data_index)
# print(data[2,:,:])
array = np.argmax(data, axis = 0)
print(array)
print(array.shape)

In [ ]:
import numpy as np
import scipy.io as io
data = np.load('../Matlab_files/dataset/originalmatdata/dataset/generatedata/label/0/detectmatrix.npy')
print(data)
# labeldata = np.load('../Matlab_files/dataset/originalmatdata/dataset/generatedata/label/20/4414.npy')
# inputdata = np.load('../Matlab_files/dataset/originalmatdata/dataset/generatedata/input/20/4414.npy')

# io.savemat('./labeldata.mat',{'labeldata':labeldata})
# io.savemat('./inputdata.mat',{'inputdata':inputdata})

In [ ]:
import numpy as np

data = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/input/82.npy')
np.set_printoptions(threshold=np.inf)
# print(data)
print(data.shape)
labeldata = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/label/1.npy')
print('labeldata:',labeldata.shape)

In [ ]:
import numpy as np
import glob
import os 

inputdataaddres = '../Matlab_files/dataset/originalmatdata/dataset/formaldata/label/*.npy'
inputdataindex = glob.glob(inputdataaddres)
print(len(inputdataindex))
for i in range(len(inputdataindex)):
    print('第',i,'个数据')
    data = np.load(inputdataindex[i])
    print('修改前维度：',data.shape)
    data = data[np.newaxis,:,:,:]
    print('修改后维度：',data.shape)
    np.save(os.path.join('../Matlab_files/dataset/dataset（2023.05.04）/label/','%d.npy'%i),data)

In [ ]:
import numpy as np

data = np.load('../Matlab_files/dataset/dataset（2023.05.04）/label/0.npy')
print(data.shape)

In [ ]:
import math
beta = 0.9
confusion = [5.6088358e+00, 2.6218030e-01, 1.4923610e-01, 3.3700470e-01,
       3.4648100e-02, 1.9298751e+00, 9.6757000e-03, 2.7778000e-03]
for i in confusion:
    weight = [((1.0 - beta) / (1.0 - math.pow(beta, i)))]
    print(weight)

In [ ]:
import torch

def cb_loss(y_pred, y_true, temp, beta, confusion):
    confusion_ = [torch.sum(temp == i) for i in range(8)]

    weight = []
    weight_dice = []
    for i, n in zip(confusion_, confusion):
        if i == 0:
            weight.append(0)
            weight_dice.append(1)
        else:
            weight_dice.append(1)
            weight.append(((1.0 - beta) / (1.0 - math.pow(beta, n))))
    weight = torch.FloatTensor(weight)
    weight_dice = torch.FloatTensor(weight_dice)
    weight = weight.to(y_pred.device)
    weight_dice = weight_dice.to(y_pred.device)

    criterion_train = torch.nn.CrossEntropyLoss(weight=weight)

    loss = criterion_train(y_pred.float(), temp.float())

    return loss

In [ ]:
confusion = [1, 2, 3, 4]
confusion_ = [5, 6, 7, 8]
for i, n in zip(confusion_, confusion):
    print('i',i)
    print('n',n)
    print(list(zip(confusion_, confusion)))

In [ ]:
import numpy as np
x = np.array([-2, 3, -9.4, 5, 0])
ex = np.sum(np.exp(x)/np.sum(np.exp(x))*np.array([0, 1, 2, 3, 4]))
print(ex)

In [ ]:
import torch
import torch.nn.functional as F
x = torch.randn(5, 5)
print(x)
target = torch.tensor([0, 2, 3, 1, 4])
one_hot = F.one_hot(target).float()
print(one_hot)
softmax = torch.exp(x)/torch.sum(torch.exp(x),dim=1).reshape(-1, 1)
print(softmax)
logsoftmax = torch.log(softmax)
print(logsoftmax)
nllloss = -torch.sum(one_hot*logsoftmax)/target.shape[0]
print(nllloss)

In [ ]:
import numpy as np
import os
import glob

def nozerosdetect(data_index):

    global m
    global s
    n = 0
    zeros_array = []
    array = np.arange((m-215), (m+1))
    
    for i in range((m-215),(m+1)):
        path = os.path.join(data_index, str(i)+'.npy')
        data = np.load(path)
        if np.all(data == 0):
            n += 1
            zeros_array.append(i)
        else:
            zeros_array.append(0)

    print('一共',n,'个全零矩阵')

    zeros_array = np.array(zeros_array)
    nozeros_array = array - zeros_array
    np.save(os.path.join(data_index, 'detectmatrix.npy'), nozeros_array)

__训练集0/1数据统计__

In [ ]:
import numpy as np
import glob
dataindex = glob.glob('../../Code/dataset_mini_batch/input/positive_input/*.npy')
num_1 = 0
num_0 = 0
for t in range(len(dataindex)):
    data = np.load(dataindex[t])
    data = np.squeeze(data)
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            for k in range(data.shape[2]):
                if data[i][j][k] == 1:
                    num_1 += 1
                else:
                    num_0 += 1
                
print(num_1)
print(num_0)

In [ ]:
import torch

a = torch.tensor([1, 2, 3])
print(a)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 定义生成器模型
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.net(x)

# 定义判别器模型
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)

# 设置超参数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 0.0002
batch_size = 128
num_epochs = 50
input_size = 100
hidden_size = 256
output_size = 784

# 加载MNIST数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 初始化生成器和判别器
generator = Generator(input_size, hidden_size, output_size).to(device)
discriminator = Discriminator(output_size, hidden_size).to(device)

# 定义损失函数和优化器
criterion = nn.BCELoss()
g_optimizer = optim.Adam(generator.parameters(), lr=lr)
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)

# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device)
        real_labels = torch.ones(images.size(0), 1).to(device)
        fake_labels = torch.zeros(images.size(0), 1).to(device)

        # 训练判别器
        outputs = discriminator(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        z = torch.randn(images.size(0), input_size).to(device)
        fake_images = generator(z)
        outputs = discriminator(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        d_loss = d_loss_real + d_loss_fake
        discriminator.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # 训练生成器
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)

        generator.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'
                  .format(epoch+1, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))

# 保存模型
torch.save(generator.state_dict(), 'generator.ckpt')
torch.save(discriminator.state_dict(), 'discriminator.ckpt')

In [ ]:
import  torch
device = 'cpu'
batch_size = 5
noise_dim = 3
fixed_noise = torch.randn((batch_size, noise_dim)).to(device)
print(fixed_noise)
print(fixed_noise.shape)

In [ ]:
import torch

img = torch.randn(28, 28)
img = img.view(-1, 784)
print(img.shape)
batch_size = img.shape[0]
print(batch_size)

In [ ]:
def artist_works():    # painting from the famous artist (real target)   
    r = 0.02 * np.random.randn(1, ART_COMPONENTS)   
    paintings = np.sin(PAINT_POINTS * np.pi) + r   
    paintings = torch.from_numpy(paintings).float()   
    return paintings



In [4]:
import time
it = iter([1, 2, 3, 4, 5])
# 循环:
while True:
    try:
        # 获得下一个值:
        x = next(it)
        print(x)
       
        time.sleep(2)
        
    except StopIteration:
        # 遇到StopIteration就退出循环
        break

1
2
3
4
5
